In [17]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [18]:
for x in db.users.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                "gender": 1,
                "full_name": {
                    "$concat": [
                        {"$toUpper": {"$substrCP": ["$name.first", 0, 1]}},
                        {
                            "$substrCP": [
                                "$name.first",
                                1,
                                {"$subtract": [{"$strLenCP": "$name.first"}, 1]},
                            ]
                        },
                        " ",
                        {"$toUpper": "$name.last"},
                    ]
                },
            }
        },
        {"$limit": 5},
    ]
):
    pprint(x)


{'full_name': 'Victor PEDERSEN', 'gender': 'male'}
{'full_name': 'Carl JACOBS', 'gender': 'male'}
{'full_name': 'Zachary LO', 'gender': 'male'}
{'full_name': 'Harvey CHAMBERS', 'gender': 'male'}
{'full_name': 'Gideon VAN DRONGELEN', 'gender': 'male'}


In [19]:
client.close()
